# Quantum Pairwise Interaction Energy (QP)
This protocol splits a selected protein residues into single amino acid fragments, and calculates the HF/RIMP2 energies between the selected amino acids and a ligand.

In [ ]:
import json
import os
import sys
import tarfile

from pdbtools import *
import requests
from datetime import datetime
from pathlib import Path

import tengu

### 0) Setup

In [ ]:
# Set our token - ensure you have exported TENGU_TOKEN in your shell; or just replace the os.getenv with your token
TOKEN = os.getenv("TENGU_TOKEN")

In [ ]:
# Define our project information
DESCRIPTION = "tengu-py demo notebook"
TAGS = ["qdx", "tengu-py", "qp", "demo"]
WORK_DIR = Path.home() / "qdx" / "tengu-py-qp-demo"
OUT_DIR = WORK_DIR / "runs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Set our inputs
SYSTEM_PDB_PATH = WORK_DIR / "test.pdb"
PROTEIN_PDB_PATH = WORK_DIR / "test_P.pdb"
LIGAND_SMILES_STR = "CCCc1ccccc1O"
LIGAND_PDB_PATH = WORK_DIR / "test_L.pdb"
MODULE_LOCK = WORK_DIR / "lock.json"

In [ ]:
# fetch datafiles
complex = list(pdb_keepcoord.keep_coordinates(pdb_fetch.fetch_structure("3HTB")))
protein = pdb_delhetatm.remove_hetatm(pdb_selchain.select_chain(complex, "A"))
ligand = pdb_rplresname.rename_residues(pdb_selresname.filter_residue_by_name(complex, "JZ4"), "JZ4", "UNL")
pdb_tidy
with open(SYSTEM_PDB_PATH, 'w') as f:
    for l in complex:
        f.write(str(l))
with open(PROTEIN_PDB_PATH, 'w') as f:
    for l in protein:
        f.write(str(l))
with open(LIGAND_PDB_PATH, 'w') as f:
    for l in ligand:
        f.write(str(l))       

### 1) Prepare Protein
We use PDB2PQR to fill in missing atoms and protonate / fill hydrogens

In [ ]:
pdb2pqr_result = client.run2(
    modules["pdb2pqr_tengu"],
    [
        PROTEIN_PDB_PATH,
    ],
    target="NIX_SSH",
    resources={"gpus": 1, "storage": 1_024_000_000, "walltime": 15},
    tags=TAGS,
)
pdb2pqr_run_id = pdb2pqr_result["module_instance_id"]
prepped_protein_id = pdb2pqr_result["output_ids"][0]
print(f"{datetime.now().time()} | Running protein prep!")

In [ ]:
with open(OUT_DIR / f"01-pdb2pqr-{pdb2pqr_run_id}.json", "w") as f:
    json.dump(pdb2pqr_result, f, default=str, indent=2)

In [ ]:
client.poll_module_instance(pdb2pqr_run_id)
client.download_object(prepped_protein_id, OUT_DIR / "01-prepped-protein.pdb")
print(f"{datetime.now().time()} | Downloaded prepped protein!")

In [ ]:
# Get our client, for calling modules and using the tengu API
client = tengu.Provider(access_token=TOKEN)

In [ ]:
# Get our latest modules as a dict[module_name, module_path]
# If a lock file exists, load it so that the run is reproducable
if MODULE_LOCK.exists():
    modules = client.load_module_paths(MODULE_LOCK)
else: 
    modules = client.get_latest_module_paths()
    client.save_module_paths(modules, MODULE_LOCK)